In [1]:
!pip install python-chess

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import chess
from collections import defaultdict
import random
import os
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from timeit import default_timer as timer
import pickle
import psutil

# 1. Setup
# 1.1 Thiết lập các đường dẫn thư mục
DATA_PATH = "/kaggle/input/chess-games/chess_games.csv"

# 1.2 Kiểm tra GPU
print("Successfulll")
print(torch.cuda.is_available())  # True nếu GPU hoạt động
print(torch.cuda.get_device_name(0))  # Tên GPU (ví dụ: T4)
print("Successfull")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/6.1 MB ? eta -:--:--

     ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/6.1 MB 12.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.5 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... done


  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=0183353a944476bad6161c170e663859c5d5946cda99d743a1ec6687b2143a26
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


Successfulll
True
Tesla T4
Successfull


In [2]:
# 2. Chuẩn bị dữ liệu
# 2.1 Đọc và lọc dataset
def load_and_filter_data(data_path, min_elo=2300):
    df = pd.read_csv(data_path)
    df = df[(df['WhiteElo'] > min_elo) & (df['BlackElo'] > min_elo)]
    print(f"Số ván cờ sau khi lọc: {len(df)}")
    if len(df) == 0:
        raise ValueError(f"Không có ván cờ nào thỏa mãn min_elo={min_elo}")
    return df

# 2.2 Tạo danh sách các ván cờ
def create_game_list(df):
    game_list = []
    invalid_games = 0
    for index, row in df.iterrows():
        pgn = row['AN']
        board = chess.Board()
        moves = []
        tokens = pgn.split()
        for token in tokens:
            if token in ['1-0', '0-1', '1/2-1/2']:
                break
            if (token[0].isdigit() or token in ['...', '{', '}', '[%eval', '[%clk'] or
                any(s in token for s in ['?!', '??', '!?', '!', '?', '#'])):
                continue
            try:
                move_obj = board.parse_san(token)
                moves.append(move_obj.uci())
                board.push(move_obj)
            except Exception as e:
                invalid_games += 1
                break
        if moves:
            game_list.append(" ".join(moves))
    print(f"Số ván cờ trong game_list: {len(game_list)}")
    print(f"Số ván cờ bị bỏ qua do lỗi: {invalid_games}")
    return game_list

# 2.3 Tạo cặp (chuỗi nước đi, nước đi tiếp theo)
def create_move_sequences(game_list, max_moves=30, save_path="/kaggle/working/chess_data.pkl", mapping_path="/kaggle/working/move_mappings.pkl"):
    print(f"RAM available: {psutil.virtual_memory().available / 1024**3:.2f} GB")
    move_to_idx = defaultdict(lambda: len(move_to_idx))
    X, y = [], []
    batch_size = 100000
    batch_count = 0
    batch_files = []
    
    for pgn in game_list:
        if not pgn.strip():
            continue
        moves = pgn.split()
        if not moves:
            continue
        for i in range(len(moves)):
            sequence = moves[max(0, i - max_moves):i]
            sequence_idx = [move_to_idx[move] for move in sequence]
            padded_sequence = [0] * (max_moves - len(sequence_idx)) + sequence_idx
            X.append(padded_sequence)
            y.append(move_to_idx[moves[i]])
            
            if len(X) >= batch_size:
                data = {
                    'X': np.array(X, dtype=np.int32),
                    'y': np.array(y, dtype=np.int32),
                    'move_to_idx': dict(move_to_idx),
                    'idx_to_move': {v: k for k, v in move_to_idx.items()}
                }
                batch_path = f"{save_path}_batch_{batch_count}.pkl"
                with open(batch_path, 'wb') as f:
                    pickle.dump(data, f)
                print(f"Đã lưu batch {batch_count} tại: {batch_path}")
                batch_files.append(batch_path)
                X, y = [], []
                batch_count += 1
    
    # Lưu batch cuối nếu còn
    if X:
        data = {
            'X': np.array(X, dtype=np.int32),
            'y': np.array(y, dtype=np.int32),
            'move_to_idx': dict(move_to_idx),
            'idx_to_move': {v: k for k, v in move_to_idx.items()}
        }
        batch_path = save_path
        with open(batch_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"Đã lưu batch cuối tại: {batch_path}")
        batch_files.append(batch_path)
    
    # Lưu toàn bộ move_to_idx và idx_to_move vào file riêng
    mappings = {
        'move_to_idx': dict(move_to_idx),
        'idx_to_move': {v: k for k, v in move_to_idx.items()}
    }
    with open(mapping_path, 'wb') as f:
        pickle.dump(mappings, f)
    print(f"Đã lưu toàn bộ move_to_idx và idx_to_move tại: {mapping_path}")
    
    # Gộp tất cả batch vào X, y
    X_all, y_all = [], []
    for batch_path in batch_files:
        with open(batch_path, 'rb') as f:
            data = pickle.load(f)
            X_all.extend(data['X'].tolist())
            y_all.extend(data['y'].tolist())
    
    print(f"Tổng số mẫu: {len(X_all)}")
    return np.array(X_all, dtype=np.int32), np.array(y_all, dtype=np.int32), dict(move_to_idx), dict(mappings['idx_to_move'])

# 2.4 Chia dữ liệu
def split_data(X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

# 2.5 Tạo Dataset và DataLoader
class ChessDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return torch.tensor(self.X[index], dtype=torch.long), torch.tensor(self.y[index], dtype=torch.long)

# 2.6 Chuẩn bị dữ liệu
def prepare_data(data_path, pkl_path="/kaggle/working/chess_data.pkl", mapping_path="/kaggle/working/move_mappings.pkl"):
    print("Tạo dữ liệu mới và ghi đè file .pkl...")
    df = load_and_filter_data(data_path)
    game_list = create_game_list(df)
    X, y, move_to_idx, idx_to_move = create_move_sequences(game_list, save_path=pkl_path, mapping_path=mapping_path)

    # In move_to_idx và idx_to_move (chỉ in 5 mục đầu tiên để tránh quá dài)
    print("\nmove_to_idx (5 mục đầu tiên):")
    print(dict(list(move_to_idx.items())[:5]))
    print(f"Tổng số mục trong move_to_idx: {len(move_to_idx)}")
    print("\nidx_to_move (5 mục đầu tiên):")
    print(dict(list(idx_to_move.items())[:5]))
    print(f"Tổng số mục trong idx_to_move: {len(idx_to_move)}")
    
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    train_dataset = ChessDataset(X_train, y_train)
    test_dataset = ChessDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"Số lượng mini-batches train: {len(train_loader)}")
    print(f"Kích thước của tập train: {len(train_loader.dataset)}")
    print(f"Kích thước của tập test: {len(test_loader.dataset)}")
    print("Successfullll !!!!")
    
    return train_loader, test_loader, move_to_idx, idx_to_move

# Gọi hàm chuẩn bị dữ liệu
train_loader, test_loader, move_to_idx, idx_to_move = prepare_data(DATA_PATH)

Tạo dữ liệu mới và ghi đè file .pkl...


Số ván cờ sau khi lọc: 57395


Số ván cờ trong game_list: 57326
Số ván cờ bị bỏ qua do lỗi: 8015
RAM available: 25.10 GB


Đã lưu batch 0 tại: /kaggle/working/chess_data.pkl_batch_0.pkl


Đã lưu batch 1 tại: /kaggle/working/chess_data.pkl_batch_1.pkl


...

Đã lưu batch 41 tại: /kaggle/working/chess_data.pkl_batch_41.pkl


Đã lưu batch cuối tại: /kaggle/working/chess_data.pkl
Đã lưu toàn bộ move_to_idx và idx_to_move tại: /kaggle/working/move_mappings.pkl


Tổng số mẫu: 4261971



move_to_idx (5 mục đầu tiên):
{'e2e4': 0, 'c7c5': 1, 'g1f3': 2, 'd7d6': 3, 'd2d4': 4}
Tổng số mục trong move_to_idx: 1877

idx_to_move (5 mục đầu tiên):
{0: 'e2e4', 1: 'c7c5', 2: 'g1f3', 3: 'd7d6', 4: 'd2d4'}
Tổng số mục trong idx_to_move: 1877


Số lượng mini-batches train: 53275
Kích thước của tập train: 3409576
Kích thước của tập test: 852395
Successfullll !!!!


In [3]:
# 3. Model Engineering
# 3.1 Định nghĩa mô hình Transformer
class ChessMovePredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, num_heads=8, num_layers=8, max_moves=30, dropout=0.2):
        super(ChessMovePredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = nn.Parameter(torch.zeros(1, max_moves, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True, 
            dropout=dropout
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x) + self.pos_encoding
        x = self.transformer(x)
        x = x[:, -1, :]  # Lấy embedding của nước đi cuối
        x = self.fc(x)
        return x

# 3.2 Khởi tạo mô hình
vocab_size = len(move_to_idx)
model = ChessMovePredictor(vocab_size=vocab_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# In số lượng tham số
total_params = sum(p.numel() for p in model.parameters())
print(f"Tổng số tham số: {total_params:,}")
print("Successful")

Tổng số tham số: 27,158,357
Successful


In [4]:
# 4. Training the Model
# 4.1 Establish Checkpoints
checkpoint_dir = '/kaggle/working/training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# 4.2 Build Training Loop
def print_train_time(start, end):
    total_time = end - start
    print(f"Training Time: {total_time:.2f} seconds")
    return total_time

# THÊM TOP-K ACCURACY
def top_k_accuracy(output, target, k=5):
    with torch.no_grad():
        _, pred = output.topk(k, dim=1)
        correct = pred.eq(target.view(-1, 1).expand_as(pred))
        return correct.any(dim=1).float().mean().item()

def train_model(model, train_loader, test_loader, epochs, learning_rate, checkpoint_path=None):
    torch.manual_seed(42)
    train_time_start = timer()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    start_epoch = 0
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Tiếp tục từ epoch {start_epoch}")

    for epoch in range(start_epoch, epochs):
        print(f"Epoch {epoch + 1}\n------------------------------")
        
        # Training
        train_loss = 0
        train_top1_acc, train_top3_acc, train_top5_acc = 0, 0, 0
        model.train()
        for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            output = model(X)
            loss = loss_fn(output, y)
            train_loss += loss.item()
            
            # Tính Top-k Accuracy
            train_top1_acc += top_k_accuracy(output, y, k=1)
            train_top3_acc += top_k_accuracy(output, y, k=3)
            train_top5_acc += top_k_accuracy(output, y, k=5)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch % 10000 == 0:
                print(f"Looked at {batch * 64}/{len(train_loader.dataset)} samples")

        train_loss /= len(train_loader)
        train_top1_acc /= len(train_loader)
        train_top3_acc /= len(train_loader)
        train_top5_acc /= len(train_loader)
        print(f"Train Loss: {train_loss:.4f} | Top-1 Acc: {train_top1_acc:.4f} | Top-3 Acc: {train_top3_acc:.4f} | Top-5 Acc: {train_top5_acc:.4f}")

        # Testing
        test_loss = 0
        test_top1_acc, test_top3_acc, test_top5_acc = 0, 0, 0
        model.eval()
        with torch.inference_mode():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                output = model(X)
                test_loss += loss_fn(output, y).item()
                
                # Tính Top-k Accuracy
                test_top1_acc += top_k_accuracy(output, y, k=1)
                test_top3_acc += top_k_accuracy(output, y, k=3)
                test_top5_acc += top_k_accuracy(output, y, k=5)

        test_loss /= len(test_loader)
        test_top1_acc /= len(test_loader)
        test_top3_acc /= len(test_loader)
        test_top5_acc /= len(test_loader)
        print(f"Test Loss: {test_loss:.4f} | Top-1 Acc: {test_top1_acc:.4f} | Top-3 Acc: {test_top3_acc:.4f} | Top-5 Acc: {test_top5_acc:.4f}")

        # Lưu checkpoint
        checkpoint_path_save = os.path.join(checkpoint_dir, f'ckpt_epoch_{epoch + 1}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_top1_acc': train_top1_acc,
            'test_top1_acc': test_top1_acc,
        }, checkpoint_path_save)
        print(f"Đã lưu checkpoint tại: {checkpoint_path_save}")

    train_time_end = timer()
    print_train_time(train_time_start, train_time_end)

# 4.3 Chạy huấn luyện
train_model(model, train_loader, test_loader, epochs=10, learning_rate=0.0001)

# Để tiếp tục từ checkpoint (ví dụ từ epoch cụ thể):
# def load_mappings(mapping_path):
#     with open(mapping_path, 'rb') as f:
#         mappings = pickle.load(f)
#     return mappings['move_to_idx'], mappings['idx_to_move']

# # Tải ánh xạ từ file .pkl
# move_to_idx, idx_to_move = load_mappings("/kaggle/input/transformer-chessbot/move_mappings.pkl")

# # Khởi tạo mô hình với vocab_size dựa trên move_to_idx
# vocab_size = len(move_to_idx)
# model = ChessMovePredictor(vocab_size=vocab_size)
# model.to(device)
# checkpoint_to_load = '/kaggle/input/transformer-chessbot/training_checkpoints/ckpt_epoch_20.pt'
# train_model(model, train_loader, test_loader, epochs=30, learning_rate=0.0001, checkpoint_path=checkpoint_to_load)

print("Successful !")

Epoch 1
------------------------------


Looked at 0/3409576 samples


Looked at 640000/3409576 samples


Looked at 1280000/3409576 samples


Looked at 1920000/3409576 samples


...

Looked at 0/3409576 samples


Looked at 640000/3409576 samples


Looked at 1280000/3409576 samples


Looked at 1920000/3409576 samples


Looked at 2560000/3409576 samples
